In [40]:
MANEUVER = 7
def read_matrix(filename):
    # Wyciągnięcie liczby z nazwy pliku
    num_str = filename.split("_")[2].split(".")[0]
    num = int(num_str)

    # Otwarcie pliku i odczytanie zawartości
    with open("data/"+filename, 'r') as f:
        content = f.read()

    # Konwersja zawartości na listę int'ów
    colision_matrix = []
    idx = 0
    numbers = [int(x) for x in content.split()]
    for plane1 in range(num):
        colision_matrix.append([])
        for plane1_maneuver in range(MANEUVER):
            colision_matrix[plane1].append([])
            for plane2 in range(num):
                colision_matrix[plane1][plane1_maneuver].append([])
                for _ in range(MANEUVER):
                    colision_matrix[plane1][plane1_maneuver][plane2].append(numbers[idx])
                    idx += 1

    return colision_matrix, num

In [3]:
def print_matrix(matrix, size):
    for i in range(size):
        for j in range(MANEUVER):
            print(matrix[i][j])
            # for k in range(size):
                # print(matrix[i][j][k])

In [57]:
filenames = ["CM_RCP_10.txt", "CM_RCP_20.txt", "CM_RCP_30.txt", "CM_RCP_40.txt"]
cm, planes_num = read_matrix(filenames[0])
# print_matrix(cm, 10)

In [59]:
from docplex.mp.model import Model

# Tworzenie modelu
model = Model(name='Binary variables model')

# Zmienne decyzyjne
x = {(i,j): model.binary_var(name='x_{0}_{1}'.format(i,j)) for i in range(0,planes_num) for j in range(0, MANEUVER)}

# Ograniczenia
# Pierwsze ograniczenie
for i in range(0,planes_num):
    model.add_constraint(model.sum(x[(i,j)] for j in range(0,MANEUVER)) == 1)

# Drugie ograniczenie
for i in range(0,planes_num):
    for j in range(0, MANEUVER):
        for k in range(i+1,planes_num):
            model.add_constraint(model.sum((x[(i,j)]*cm[i][j][k][l] + x[(k,l)]*cm[i][j][k][l]) for l in range(0,MANEUVER)) <= 1)

# Rozwiązanie problemu
model.solve()

# Wyświetlenie wyniku
print('Status: ', model.solve_details.status)
print('Czas obliczeń: ', model.solve_details.time, 's')
print('Wartości zmiennych x:')
for i in range(0,planes_num):
    for j in range(0, MANEUVER):
        print('x_{0}_{1} = {2}'.format(i, j, x[(i,j)].solution_value))


Status:  integer optimal solution
Czas obliczeń:  0.015999999988707714 s
Wartości zmiennych x:
x_0_0 = 1.0
x_0_1 = 0
x_0_2 = 0
x_0_3 = 0
x_0_4 = 0
x_0_5 = 0
x_0_6 = 0
x_1_0 = 1.0
x_1_1 = 0
x_1_2 = 0
x_1_3 = 0
x_1_4 = 0
x_1_5 = 0
x_1_6 = 0
x_2_0 = 1.0
x_2_1 = 0
x_2_2 = 0
x_2_3 = 0
x_2_4 = 0
x_2_5 = 0
x_2_6 = 0
x_3_0 = 1.0
x_3_1 = 0
x_3_2 = 0
x_3_3 = 0
x_3_4 = 0
x_3_5 = 0
x_3_6 = 0
x_4_0 = 1.0
x_4_1 = 0
x_4_2 = 0
x_4_3 = 0
x_4_4 = 0
x_4_5 = 0
x_4_6 = 0
x_5_0 = 1.0
x_5_1 = 0
x_5_2 = 0
x_5_3 = 0
x_5_4 = 0
x_5_5 = 0
x_5_6 = 0
x_6_0 = 0
x_6_1 = 0
x_6_2 = 1.0
x_6_3 = 0
x_6_4 = 0
x_6_5 = 0
x_6_6 = 0
x_7_0 = 0
x_7_1 = 1.0
x_7_2 = 0
x_7_3 = 0
x_7_4 = 0
x_7_5 = 0
x_7_6 = 0
x_8_0 = 1.0
x_8_1 = 0
x_8_2 = 0
x_8_3 = 0
x_8_4 = 0
x_8_5 = 0
x_8_6 = 0
x_9_0 = 1.0
x_9_1 = 0
x_9_2 = 0
x_9_3 = 0
x_9_4 = 0
x_9_5 = 0
x_9_6 = 0
